In [2]:
print('hi, welcome to langchain!')

hi, welcome to langchain!


In [3]:
pip install -qU "langchain[aws]"

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.19.0 requires botocore<1.36.4,>=1.36.0, but you have botocore 1.40.25 which is incompatible.


In [4]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b."""
    return a * b

In [8]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain_community.llms.fake import FakeListLLM

# A simple tool function: calculator
def add_numbers(input: str) -> str:
    try:
        numbers = list(map(int, input.strip().split()))
        return str(sum(numbers))
    except Exception as e:
        return f"Error: {str(e)}"

# Wrap the tool in LangChain Tool
tools = [
    Tool(
        name="Adder",
        func=add_numbers,
        description="Useful for adding numbers. Input should be space-separated numbers. E.g., '2 3'"
    )
]

# Create a fake LLM that simulates outputs
fake_responses = [
    "Action: Adder\nAction Input: 2 9",
    "Final Answer: 5"
]
llm = FakeListLLM(responses=fake_responses)

# Create the agent with fake LLM and tool
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Run the agent with a prompt
result = agent.run("What is the sum of 2 and 9?")
print("\nFinal Result:", result)




> Entering new AgentExecutor chain...
Action: Adder
Action Input: 2 9
Observation: 11
Thought:Final Answer: 5

> Finished chain.

Final Result: 5


In [9]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType
from langchain_community.llms.fake import FakeListLLM
from langchain.tools import tool

# ✅ Step 1: Define your tool using the @tool decorator
@tool
def add_numbers(input: str) -> str:
    """
    Adds space-separated integers. Example: '2 3' => '5'
    """
    try:
        numbers = list(map(int, input.strip().split()))
        return str(sum(numbers))
    except Exception as e:
        return f"Error: {str(e)}"

# ✅ Step 2: Create a fake LLM that simulates a real one
fake_responses = [
    "Action: add_numbers\nAction Input: 2 3",  # The agent decides to use the tool
    "Final Answer: 5"                          # After getting the tool's response
]
llm = FakeListLLM(responses=fake_responses)

# ✅ Step 3: Initialize the agent with the tool
agent = initialize_agent(
    tools=[add_numbers],        # Pass the decorated function directly
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# ✅ Step 4: Run the agent
result = agent.run("What is 2 plus 3?")
print("\nFinal Result:", result)




> Entering new AgentExecutor chain...
Action: add_numbers
Action Input: 2 3
Observation: 5
Thought:Final Answer: 5

> Finished chain.

Final Result: 5


In [10]:
# LangChain & Agentic AI Cheatsheet

---

## 1. What is an Agent?

- An **Agent** is a system that uses an LLM **plus logic** to:
  - Interpret the LLM’s output
  - Decide what *actions* to take (e.g., call tools)
  - Loop and reason until a final answer

- LLM = The *brain* (generates text/predictions)  
- Agent logic = The *controller* (decides next step, calls tools)

---

## 2. What are Tools?

- Tools = External functions the agent can call, e.g.:
  - Calculators
  - Search engines
  - APIs
  - Document retrievers

- Decorated with `@tool` in LangChain to define usage and description

---

## 3. Why use Agent instead of just LLM?

| LLM only                    | Agent (LLM + tools + logic)                |
|-----------------------------|--------------------------------------------|
| Generates answers directly   | Dynamically chooses and calls tools        |
| No multi-step decision logic | Supports multi-step reasoning & workflows  |
| No external action           | Can call calculators, APIs, retrieval etc. |

---

## 4. AgentType in LangChain

- `ZERO_SHOT_REACT_DESCRIPTION`:  
  Zero-shot ReAct (Reason + Act) using tool descriptions; default and versatile.

- Other notable types:  
  - `OPENAI_FUNCTIONS`: Uses OpenAI function calling API  
  - `STRUCTURED_CHAT_ZERO_SHOT_REACT`: Structured multi-turn chat  
  - `CHAT_CONVERSATIONAL_REACT_DESCRIPTION`: Chat-oriented ReAct  
  - `SELF_ASK_WITH_SEARCH`: Self-asking with search tool  
  - `PLAN_AND_EXECUTE`: Plan all steps, then execute sequentially

---

## 5. What is Agentic AI?

- AI systems that act **autonomously** to achieve goals by:
  - Planning
  - Taking actions (calling tools/APIs)
  - Adapting dynamically

- LangChain agents are a simple, practical example of agentic AI.

---

## 6. RAG vs Agent

| Aspect                | RAG (Retrieval-Augmented Generation)          | Agent                                        |
|-----------------------|-----------------------------------------------|----------------------------------------------|
| Uses external data?   | Yes, via retrieval from vector DB              | Yes, plus other tools/APIs                    |
| Multi-step reasoning? | No (single pass)                               | Yes (loops, multiple steps)                   |
| Dynamic tool calls?   | No                                            | Yes                                           |
| Best for             | Static document Q&A                            | Complex workflows & dynamic tool usage        |

---

## 7. Example Hybrid Setup: Agent + Retriever Tool

```python
from langchain.tools import tool

@tool
def document_search_tool(query: str) -> str:
    results = retriever.get_relevant_documents(query)
    return "\n\n".join([doc.page_content for doc in results])

@tool
def calculator_tool(expression: str) -> str:
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

tools = [document_search_tool, calculator_tool]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run("What is the refund policy? Also, calculate 12 * 15.")
print(response)


SyntaxError: invalid character '’' (U+2019) (1663093016.py, line 8)